In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
dfraw = pd.read_csv("C:/Users/Don/Desktop/DATA 301/Project/FiresNEW.csv", encoding = 'ISO-8859-1', low_memory=False)
df1 = dfraw.drop(columns=["FOD_ID","FPA_ID","SOURCE_SYSTEM_TYPE","SOURCE_SYSTEM","NWCG_REPORTING_AGENCY","NWCG_REPORTING_UNIT_ID","NWCG_REPORTING_UNIT_NAME","SOURCE_REPORTING_UNIT","SOURCE_REPORTING_UNIT_NAME","LOCAL_FIRE_REPORT_ID","LOCAL_INCIDENT_ID","FIRE_CODE","FIRE_NAME","ICS_209_PLUS_INCIDENT_JOIN_ID","ICS_209_PLUS_COMPLEX_JOIN_ID","MTBS_ID","MTBS_FIRE_NAME","COMPLEX_NAME","DISCOVERY_DATE","DISCOVERY_DOY","DISCOVERY_TIME","NWCG_GENERAL_CAUSE","NWCG_CAUSE_AGE_CATEGORY","CONT_DATE","CONT_DOY","CONT_TIME","LATITUDE","LONGITUDE","OWNER_DESCR","COUNTY","FIPS_CODE","FIPS_NAME"])
df1 = df1.drop(df1[df1.NWCG_CAUSE_CLASSIFICATION == 'Human'].index)
df1 = df1.drop(df1[df1.NWCG_CAUSE_CLASSIFICATION == 'Missing data/not specified/undetermined'].index)
df1 = df1.dropna()
df1 = df1.drop(df1[df1.FIRE_SIZE_CLASS == 'A'].index)
df1 = df1.drop(columns=['NWCG_CAUSE_CLASSIFICATION'])

In [4]:
dfstate = pd.read_csv("C:/Users/Don/Desktop/DATA 301/Project/csvData.csv")
dfstate = (
    dfstate.drop(columns='Abbrev')
    .rename(columns={'Code':'STATE'})
)

In [5]:
dftemp = pd.read_csv("C:/Users/Don/Desktop/DATA 301/Project/tavg.csv")
f = lambda dftemp : dftemp['Location'].split("CD")
dftemp['Location'] = dftemp.apply(f, axis=1)
g = lambda dftemp : dftemp['Location'].pop(0)
dftemp['Location'] = dftemp.apply(g, axis=1)
dftemp = dftemp.reset_index()
dftemp = dftemp.drop(columns=['Location ID','Value','Rank','Anomaly (1901-2000 base period)'])
dftemp['Celsius'] = (dftemp['1901-2000 Mean']-32)*5/9
dftemp['Celsius'] = dftemp['Celsius'].round(1)
dftemp = dftemp.drop(columns=['1901-2000 Mean'])
dftemp = dftemp.drop(dftemp[dftemp.Date % 10 != 7].index)
dftemp['Date'] = dftemp['Date']//100
dftemp = pd.DataFrame(dftemp.groupby(['Location','Date','Celsius']).sum())
dftemp = dftemp.reset_index(inplace=True)